In [98]:
import torch
import numpy as np
import time
import torch.nn as nn
device = "cuda" if torch.cuda.is_available() else "cpu"

In [192]:
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2
n_embed = 384
n_layers = 6
dropout = 0.2

In [1]:
with open("book.txt","r",encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '"', '#', '&', '(', ')', '*', ',', '-', '.', '0', '1', '2', '4', '5', '6', '7', '8', '9', ':', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'W', 'Y', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
68


In [ ]:
str_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_str = {i:ch for i,ch in enumerate(chars)}

3

In [ ]:
# 定义两个方法，将一个列表中的所有字母/数字都转化成数字/字母
encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_str[i] for i in l]) # 按照分隔符""将字符连接起来
decode2 = lambda l: [int_to_str[i] for i in l]
en_data = encode("hello")
en_data
de_data = decode(en_data)
de_data

'hello'

In [22]:
test1 = encode("hello")
print(test1)
test2 = decode(test1)
print(test2)
test3 =decode2(test1)
print(test3)

[52, 49, 56, 56, 59]
hello
['h', 'e', 'l', 'l', 'o']


In [72]:
data = torch.tensor(
    encode(text),
    dtype = torch.long
)
data[:100]

tensor([39, 52, 49,  1, 36, 61, 59, 54, 49, 47, 63,  1, 27, 64, 63, 49, 58, 46,
        49, 61, 51,  1, 49, 22, 59, 59, 55,  1, 59, 50,  1, 23, 45, 60, 63, 45,
        53, 58, 62,  1, 59, 50,  1, 62, 59, 64, 56, 62,  0,  1,  1,  1,  1,  0,
        39, 52, 53, 62,  1, 49, 46, 59, 59, 55,  1, 53, 62,  1, 50, 59, 61,  1,
        63, 52, 49,  1, 64, 62, 49,  1, 59, 50,  1, 45, 58, 67, 59, 58, 49,  1,
        45, 58, 67, 66, 52, 49, 61, 49,  1, 53])

In [73]:
len(text)

1110

In [75]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]
print(len(train_data))
print(len(val_data))

888
222


In [74]:
block_size = 8
batch_size = 4

In [175]:
length = len(train_data)

def get_batch(split):
    dataset = train_data if split == "train" else val_data
    ix = torch.randint(0, len(dataset) - block_size, (batch_size,)) # 总共是几个batch
    # a = [dataset[i:i+block_size] for i in ix] # 将每个batch构建成列表
    x = torch.stack([dataset[i:i+block_size] for i in ix])  # 将每个batch中的数据进行堆叠，将列表中的元素进行堆叠
    y = torch.stack([dataset[i+1:i+1+block_size] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

In [173]:
x,y = get_batch('train')
print(f"input:\n{x}")
print(f"target:\n{y}")

[tensor([66,  1, 42, 59, 61, 55,  0,  0]), tensor([ 1, 23, 59, 60, 67, 61, 53, 51]), tensor([ 1, 38, 35, 40, 32, 38,  0,  0]), tensor([61, 67,  1, 66, 52, 49, 61, 49])]
input:
tensor([[66,  1, 42, 59, 61, 55,  0,  0],
        [ 1, 23, 59, 60, 67, 61, 53, 51],
        [ 1, 38, 35, 40, 32, 38,  0,  0],
        [61, 67,  1, 66, 52, 49, 61, 49]])
target:
tensor([[ 1, 42, 59, 61, 55,  0,  0,  1],
        [23, 59, 60, 67, 61, 53, 51, 52],
        [38, 35, 40, 32, 38,  0,  0,  0],
        [67,  1, 66, 52, 49, 61, 49,  1]])
input:
tensor([[66,  1, 42, 59, 61, 55,  0,  0],
        [ 1, 23, 59, 60, 67, 61, 53, 51],
        [ 1, 38, 35, 40, 32, 38,  0,  0],
        [61, 67,  1, 66, 52, 49, 61, 49]], device='cuda:0')
target:
tensor([[ 1, 42, 59, 61, 55,  0,  0,  1],
        [23, 59, 60, 67, 61, 53, 51, 52],
        [38, 35, 40, 32, 38,  0,  0,  0],
        [67,  1, 66, 52, 49, 61, 49,  1]], device='cuda:0')


In [96]:
# 取一个batch，查看输入和目标是什么
x = train_data[:block_size]
y = train_data[1: block_size + 1]
for i in range(block_size-1):
    context = x[0:i + 1]
    target = y[i]
    print(f"when the input is {context}, the target is {target}")

when the input is tensor([39]), the target is 52
when the input is tensor([39, 52]), the target is 49
when the input is tensor([39, 52, 49]), the target is 1
when the input is tensor([39, 52, 49,  1]), the target is 36
when the input is tensor([39, 52, 49,  1, 36]), the target is 61
when the input is tensor([39, 52, 49,  1, 36, 61]), the target is 59
when the input is tensor([39, 52, 49,  1, 36, 61, 59]), the target is 54


In [102]:
a = torch.randint(0,4,(2,3,4))
print(a.shape)
w,t,c = a.shape
b = a.view(t,c,w)
print(b.shape)

torch.Size([2, 3, 4])
torch.Size([3, 4, 2])


In [103]:
import torch.nn.functional as F

In [185]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train","val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split=split)
            _, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    """
    one head of self-attention
    """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        # 将tril注册为一个缓冲区，他们是模型的一部分，用作mask，但不会在训练中参与梯度更新
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))# 其实就是建立一个下三角矩阵

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # input of size: (batch, time-step, channels)
        # output of size: (batch, time-step, head_size)
        _,T,_ = x.shape
        k = self.key(x) # shape:(B,T,hs)
        q = self.query(x) # shape:(B,T,hs)
        # compute attention scores
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,hs) * (B,hs,T) -> (B,T,T)
        # masked_fill:根据mask来填充张量中的某些位置，若是对应位置为True,则将该位置填充为指定的值
        # torch.tril():将某个张量转化为下三角矩阵，上三角部分置为0
        # 用-inf替换掉wei中与tril的上三角部分对应的元素
        wei = wei.masked_fill(
            self.tril[:T, :T] == 0,
            float('-inf')
        )
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

In [190]:
class MultiHeadAttention(nn.Module):
    """
    multiple heads of self-attention in parallel
    """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(out)
        return out

In [ ]:
class FeedForward(nn.Module):
    """
    a simple linear layer followed by a non-linearity
    """
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)


In [191]:
class Block(nn.Module):
    """
    Transformer Block: communication followed by computation
    """
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffd(x)
        x = self.ln2(x + y)
        return x

*是解包操作，将列表中的元素一个个进行解决出来，做为单独的参数传给函数
nn.Sequential()并不接受列表的输入，需要将其解包，然后一个个输入，一个个进行构建

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
    
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, idx, targets=None):
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)
            logits = logits[:, -1, :] # take the last time step
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

In [126]:
vocab_size = len(chars)
model = BigramLanguageModel(vocab_size=vocab_size)
model = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


mc
ubsK(-"kGEsf[M:.hC9(9U1uUC5-"J5mYtOY1sGoM#yA,OB"ehMC4D.86"LftTpY2okpBBy#k6rfRpp(9(2NC#D&n
mM1[yD4dGM
tHY&n 2)[1ymTNRe41t9kh((-PNbsK(&Ou[45iEj hAd[Ur85,(tdOIyb1tISNHrsY4d2F21wauJ4dYAlYiNoudJ5:Us#
7J-2el6"lAopEcFskkun ElK28WT2PcC"#dl8WT&bwMBk[YujaF.UsM-ga[Yku  b,C1*f5Kg E)o g8,IT4NbhMC:k698kyF5r5TgcuPuB&mR)wSuIuI7MCb7
LLwKLw9G2##U0[-Ls(4-*wIH6h EJ)#0LA02oO,bPAYp(-s0&UpY*e5u1
m4hjyadoJyisK hNW6p#&,J4dD5d(
u:S0(]hr.low7J*6-&#nyuLm fpFORP[TL*rh(M&fi#J4K)w:&MAyUhalsllopiO7HOK0*wpY6r#.fs&yJSD:lDnteD


Adamw:
Adamw is a modification of the Adam optimizer that add weight decay to the parameter updates.
This helps to regularize the model and improve generalization performance.

In [186]:
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr = learning_rate
)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step{iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)

    optimizer.zero_grad(set_to_none=True) #优化之前将所有的梯度置为None
    loss.backward() # 反向传播误差
    optimizer.step() # 正向传递信息
print(loss.item())

step0, train loss: 2.1638, val loss: 3.2442
step250, train loss: 2.1405, val loss: 3.2682
step500, train loss: 2.1142, val loss: 3.3173
step750, train loss: 2.1446, val loss: 3.3049
step1000, train loss: 2.1486, val loss: 3.2719
step1250, train loss: 2.1537, val loss: 3.2518
step1500, train loss: 2.1577, val loss: 3.3656
step1750, train loss: 2.1348, val loss: 3.2823
step2000, train loss: 2.1454, val loss: 3.2999
step2250, train loss: 2.1358, val loss: 3.2458
step2500, train loss: 2.1307, val loss: 3.3595
step2750, train loss: 2.1267, val loss: 3.3401
step3000, train loss: 2.1191, val loss: 3.3664
step3250, train loss: 2.1383, val loss: 3.2924
step3500, train loss: 2.1225, val loss: 3.2807
step3750, train loss: 2.1300, val loss: 3.3833
step4000, train loss: 2.1081, val loss: 3.2922
step4250, train loss: 2.1271, val loss: 3.2973
step4500, train loss: 2.1219, val loss: 3.3225
step4750, train loss: 2.1347, val loss: 3.3172
step5000, train loss: 2.1313, val loss: 3.4493
step5250, train los